# Assignment Number: 7
# Names: Francis Kwame Segbe & Sanyuja Kadam

In [77]:
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [78]:
library(caret)

Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


In [82]:
# import datasets
raw_train <- read.csv('mnist_train.csv',header = FALSE)
raw_test <- read.csv('mnist_test.csv', header = FALSE)

In [83]:
train <- cbind(raw_train[,1], round(raw_train[,2:ncol(raw_train)]/255))
test <- cbind(raw_test[,1], round(raw_test[,2:ncol(raw_test)]/255))

# Question 1

In [84]:
# priors is first column class(intensity) divided by number of rows
prior_table = round(table(train[,1])/ nrow(train),3)
prior_table = as.data.frame(prior_table)
colnames(prior_table) <- c("Class", "Priors")
prior_table

Class,Priors
<fct>,<dbl>
0,0.099
1,0.112
2,0.099
3,0.102
4,0.097
5,0.090
6,0.099
7,0.104
8,0.098


# Question Number 2

In [85]:
# seperate by 10 categories
categories <- lapply(0:9, function(x) train[train[, 1] == x, -1])
likelyhood <- lapply(1:5, function(k) lapply (categories, function(x) (colSums(x) + k)/(nrow(x) + k * 2)))

k = cbind(c(round(1 - likelyhood[[1]][[6]][682], 3), round(1 - likelyhood[[5]][[6]][682], 3)), 
                 c(round(likelyhood[[1]][[10]][772], 3), round(likelyhood[[5]][[10]][772], 3))) 
                                              
rownames(k) = c('K = 1', 'K = 5')
colnames(k) = c('P(F682=0|class=5)', 'P(F772=1|class=9)')                                              

k

,P(F682=0|class=5),P(F772=1|class=9)
K = 1,0.85,0.001
K = 5,0.85,0.002


# Question 3

In [88]:
# similiar to soln of final 

MaxPosteriori <- apply(test[,-1], MARGIN = 1, function(r){
    # cycle Ks
    lapply(1:5, function(k){
        # cycle through and apply function described in paragraph above
        # use log to avoid underflow 
        lapply(1:10, function(class){
            sum(log(likelyhood[[k]][[class]][c(which(r == 1))])) + 
            sum(log(1-likelyhood[[k]][[class]][c(which(r == 0))])) + 
            log(prior_table$Priors[[class]])}
              )
    })
})

In [90]:
# using function above.
# input format should be [1][k][target class]
# here target class 6 translates to pixel being a 5 in data
# here target class 8 translates to pixel being a 7 in data

k1 <- cbind(c(round(MaxPosteriori[[1]][[1]][[6]], 3), round(MaxPosteriori[[1]][[5]][[6]], 3)), 
                  c(round(MaxPosteriori[[1]][[1]][[8]], 3), round(MaxPosteriori[[1]][[5]][[8]], 3)))

colnames(k1) = c('P(class = 5|f1, f2, ..., f784)', 'P(class = 7|f1, f2, ..., f784)')
rownames(k1) = c('K = 1', 'K = 5')

k1

,"P(class = 5|f1, f2, ..., f784)","P(class = 7|f1, f2, ..., f784)"
K = 1,-206.095,-114.630
K = 5,-205.915,-115.022


# Question 4

In [91]:
# go through every k. For that k extract max values as this owuld be prediciton for pixel ...
predictions <- lapply(1:5, function(k){
    lapply(1: nrow(test), function(x) which.max(MaxPosteriori[[x]][[k]]) -1)
           })
    
# make confusion matrix. Got this from online and modified 
Confusion = lapply(1:5, function(k) confusionMatrix(as.factor(unlist(predictions[k])), as.factor(test[,1])) )
                   
# create table to anser
k2 <- lapply(1:5, function(k) Confusion[[k]]$overall[[1]])
print(k2)

[[1]]
[1] 0.8426

[[2]]
[1] 0.8425

[[3]]
[1] 0.8419

[[4]]
[1] 0.8417

[[5]]
[1] 0.8412



In [92]:
Predicts_k <- max_indices-1
true_labels <- Test_raw[, 1]
accuracy <- sapply(1:ncol(Predicts_k), function(col_idx) {
  correct_predictions <- sum(true_labels == Predicts_k[, col_idx])
  total_predictions <- length(true_labels)
  return(correct_predictions / total_predictions)
})
print(accuracy)

[1] 0.8426 0.8425 0.8419 0.8417 0.8412


# Question 5

In [93]:
# best k is 1 so get confusion matrix 1 
Confusion <- t(Confusion[[1]]$table)
round(Confusion / matrix(c(rep(colSums(Confusion),10)), byrow=TRUE, nrow=10) * 100,2)

         Prediction
Reference     0     1     2     3     4     5     6     7     8     9
        0 91.68  0.00  0.32  0.38  0.10  6.18  2.19  0.11  1.71  0.00
        1  0.00 88.80  0.64  0.47  0.00  1.09  0.42  0.00  2.52  0.00
        2  1.77  1.14 88.98  3.00  2.20  0.61  2.93  1.70  5.73  0.18
        3  0.42  1.39  3.81 78.71  0.10  3.64  0.73  1.38  4.43  1.68
        4  0.21  0.57  0.42  0.00 83.47  0.24  1.67  0.21  1.21 12.26
        5  1.66  0.90  0.53  9.94  2.51 79.39  1.88  0.85  2.41  2.20
        6  1.77  1.14  1.59  0.09  1.46  4.48 89.13  0.00  0.70  0.00
        7  0.21  2.70  1.80  0.28  1.78  0.00  0.00 92.01  2.52  5.91
        8  1.04  2.29  1.27  6.38  1.67  3.27  1.04  0.64 76.66  3.09
        9  1.25  1.06  0.64  0.75  6.69  1.09  0.00  3.09  2.11 74.69